## Q1.

Add a __setitem__ to the python linked list implementation from the lecture (this past wednesday).

In [2]:
#your code here
from doctest import run_docstring_examples as dtest
import numbers
import reprlib
class LL:
    """
    >>> A = LL()  
    >>> A[0]
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
    >>> A.insert_front(1)
    >>> A[0]
    1
    >>> A.insert_back(2)
    >>> A[1]
    2
    >>> A
    LL([1,...])
    >>> myll = LL.from_components([1,2])
    >>> myll[1]
    1
    >>> len(myll)
    2
    >>> myll[2]
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
    >>> myll[0:1]
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
    """
    @classmethod
    def from_components(cls, components):
        inst = cls(components[0])
        for c in components[1:]:
            inst.insert_front(c)
        return inst
        
    def __init__(self, head=None):
        if head is None:
            self._headNode = None
        else:
            self._headNode = [head, None]
            
    def insert_front(self, element):
        new_node = [element, None]
        new_node[1] = self._headNode
        self._headNode = new_node
        
    def insert_back(self, element):
        new_node = [element, None]
        curr_ptr = self._headNode
        while curr_ptr[1] is not None:
            curr_ptr = curr_ptr[1]
        curr_ptr[1]= new_node
        
    def __repr__(self):
        class_name = type(self).__name__
        if len(self)==0:
            components=""
        else:
            components = reprlib.repr(self[0])
        return '{}([{},...])'.format(class_name,components)


    def __len__(self):
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            return 0
        while 1:
            count = count + 1
            if curr_ptr[1] is None:
                break
            curr_ptr = curr_ptr[1]
        return count    
    
    def __getitem__(self, index):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    return curr_ptr[0]
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))
            
    def __setitem__(self, index, element):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:  #if the curr_ptr is none, then the linked list is empty
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                #if the current index is the target one, then change its value to the new element
                if index == count:
                    curr_ptr[0] = element
                    return
                #if the current index is not the target one and it's the last one of the linked list,
                #then raise a IndexError
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            #if index is not integer, raise a TypeError
            msg = '{class_name} indices must be integers'  
            raise TypeError(msg.format(class_name=class_name))
            

In [3]:
#test my setitem function
A = LL()
A.insert_front(1)
A.insert_back(2)
print("old_element:" + str(A[1]))
A[1]= 3
print("new_element:" + str(A[1]))

old_element:2
new_element:3


## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [4]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() ) #create a random python data
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [5]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [6]:
g = make_data(5, 10)
list(g)

[999999996.3260335,
 1000000001.0862633,
 1000000000.7680495,
 999999998.093252,
 999999999.5335846,
 999999998.532835,
 1000000000.3266121,
 1000000004.5909083,
 1000000001.5212009,
 1000000000.0033947,
 1000000000.6106484]

In [7]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)[:10]

<class 'generator'>


[999999996.5634227,
 999999996.6407385,
 999999997.1509119,
 999999999.1424103,
 999999999.4641758,
 999999998.643685,
 999999999.161778,
 999999999.4390455,
 999999999.6230265,
 999999999.8834962]

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

Formula to calculate the sample variance:<br>
Initialize $M_1 = x_1$ and $S_1 = 0$ <br>
For subsequent x's, use the recurrence formulas<br>

$M_k = M_{k-1}+ (x_k - M_{k-1})/k$<br>
$S_k = S_{k-1} + (x_k - M_{k-1})*(x_k – M_k).$

In [16]:
# your code here
import math
def online_mean_dev(iterator):
    n = 0
    mu = 0 
    dev_accum = 0
    #stddev = 0
    for value in iterator:
        n += 1
        #use the formula in the last cell
        mu_new = mu + (value - mu)/n
        dev_accum = dev_accum + (value - mu) * (value - mu_new)
        mu = mu_new
        if n > 1:
            stddev = math.sqrt(dev_accum/(n-1))
        else:
            stddev = 0
        yield (n, value, mu, stddev) 

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [11]:
data_with_stats = online_mean_dev(make_data(5, 100000))
#list(data_with_stats)[:10]

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.
<font color=blue> (mean+-level*$\sigma$)

In [12]:
#your code here
def is_ok(level, t):
    n, value, mu, stddev = t
    if stddev == 0:
        return True
    if math.fabs((value-mu)) / stddev < level:
        return True
    else: return False

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [13]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
print (pred)
anomalies = filterfalse(pred, data_with_stats)

<function <lambda> at 0x1040cc0d0>


We materialize the anomalies...

In [14]:
list(anomalies)#materialize

[(902, 1000000014.109446, 1000000000.0091741, 2.78624427599756),
 (8733, 1000000015.3192644, 1000000000.0271835, 2.9237170845803027),
 (9173, 999999984.9808483, 1000000000.0274365, 2.9128825502234856),
 (11963, 999999985.0426213, 1000000000.0199275, 2.8845438164629904),
 (14881, 999999985.5012885, 999999999.9993515, 2.882799539799625),
 (15450, 999999982.4162916, 999999999.9945586, 2.8917084949059384),
 (20816, 1000000018.7260305, 999999999.9969579, 2.8980509737612636),
 (22473, 1000000016.8269157, 999999999.9915435, 2.907261747642284),
 (23781, 1000000016.489669, 999999999.995187, 2.905536963738286),
 (29442, 1000000014.8428655, 1000000000.0006701, 2.908022208840012),
 (29814, 999999985.3596138, 1000000000.0027888, 2.909097476571968),
 (30635, 999999985.3822982, 1000000000.0030938, 2.903532532916623),
 (30803, 999999981.9434892, 1000000000.0024455, 2.9046474393976767),
 (32376, 1000000014.6008277, 1000000000.0054848, 2.9012966952037167),
 (32489, 999999984.6010219, 1000000000.0031742,

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).
<font color=blue> one iterator is resumed by another one and form a pipeline.